In [1]:
import re # regular expression
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
#converting a collection of text documents into a numerical representation
from sklearn.model_selection import train_test_split

from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
#Tokenization is the process of dividing a text into smaller units
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
#A layer which learns a position embedding for inputs sequences.

from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# A common solution for SSL certificate_verify_failed errors.

In [2]:
data = pd.read_csv("C:/Users/Rohan C/Dropbox/PC/Downloads/RNN.csv")

#data = pd.read_csv('RNN.csv')
# Keeping only the neccessary columns
data = data[['Text','Sentiment']]

In [3]:
data.head(10)

,Text,Sentiment
0,The recipe was easy to follow and resulted in ...,Positive
1,The flavors in this meal are simply exquisite!,Positive
2,I absolutely loved how tender and juicy the me...,Positive
3,"The presentation of the dish was so beautiful,...",Positive
4,The aroma of the spices filled the kitchen and...,Positive
5,This cookbook has become my go-to resource for...,Positive
6,The chef's attention to detail really shines t...,Positive
7,I never knew cooking could be this enjoyable u...,Positive
8,The ingredients used were fresh and of high qu...,Positive
9,The cooking technique used in this recipe elev...,Positive


In [4]:
data = data[data.Sentiment != "Neutral"]
data['Text'] = data['Text'].apply(lambda x: x.lower())
data['Text'] = data['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X)

In [5]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print('Shape of training samples:',X_train.shape,Y_train.shape)
print('Shape of testing samples:',X_test.shape,Y_test.shape)

Shape of training samples: (138, 36) (138, 2)
Shape of testing samples: (69, 36) (69, 2)


In [6]:
model = Sequential()
#model.add(Embedding(max_fatures, 128 ,input_length = X.shape[1], dropout=0.2))
model.add(Embedding(max_fatures, 128 ,input_length = X.shape[1]))
model.add(LSTM(128))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 36, 128)           256000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 387842 (1.48 MB)
Trainable params: 387842 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:
batch_size = 16
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
9/9 - 0s - loss: 7.5622e-05 - accuracy: 1.0000 - 181ms/epoch - 20ms/step
Epoch 2/20
9/9 - 0s - loss: 5.4854e-05 - accuracy: 1.0000 - 164ms/epoch - 18ms/step
Epoch 3/20
9/9 - 0s - loss: 4.1960e-05 - accuracy: 1.0000 - 165ms/epoch - 18ms/step
Epoch 4/20
9/9 - 0s - loss: 9.3429e-05 - accuracy: 1.0000 - 169ms/epoch - 19ms/step
Epoch 5/20
9/9 - 0s - loss: 5.2517e-05 - accuracy: 1.0000 - 161ms/epoch - 18ms/step
Epoch 6/20
9/9 - 0s - loss: 4.4754e-05 - accuracy: 1.0000 - 154ms/epoch - 17ms/step
Epoch 7/20
9/9 - 0s - loss: 3.5363e-05 - accuracy: 1.0000 - 150ms/epoch - 17ms/step
Epoch 8/20
9/9 - 0s - loss: 3.1756e-05 - accuracy: 1.0000 - 152ms/epoch - 17ms/step
Epoch 9/20
9/9 - 0s - loss: 3.2284e-05 - accuracy: 1.0000 - 142ms/epoch - 16ms/step
Epoch 10/20
9/9 - 0s - loss: 3.0349e-05 - accuracy: 1.0000 - 147ms/epoch - 16ms/step
Epoch 11/20
9/9 - 0s - loss: 2.7669e-05 - accuracy: 1.0000 - 184ms/epoch - 20ms/step
Epoch 12/20
9/9 - 0s - loss: 2.7813e-05 - accuracy: 1.0000 - 153ms/epoch -

In [31]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Accuracy: %.2f" % (acc))

5/5 - 0s - loss: 1.9762 - accuracy: 0.7536 - 55ms/epoch - 11ms/step
Score: 1.98
Accuracy: 0.75


In [32]:
text = 'Yummy'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 21ms/step
Prediction value: [0.46619546 0.5338046 ]
Positive


In [33]:
text = 'Mouthwatering flavours!'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 22ms/step
Prediction value: [5.5755285e-04 9.9944240e-01]
Positive


In [34]:
text = 'Horrible taste'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 21ms/step
Prediction value: [9.9998713e-01 1.2932285e-05]
Negative


In [35]:
text = 'Disappointing flavor.'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 24ms/step
Prediction value: [9.9996161e-01 3.8436898e-05]
Negative


In [36]:
text = 'The recipe was easy to follow and resulted in a delicious dish.'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 20ms/step
Prediction value: [2.3204578e-07 9.9999976e-01]
Positive


In [37]:
text = 'The dish lacked flavor and was quite bland, despite following the recipe closely.'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 25ms/step
Prediction value: [1.0000000e+00 4.9796967e-09]
Negative


In [38]:
text = 'Taste was awful'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 27ms/step
Prediction value: [9.999105e-01 8.950241e-05]
Negative


In [39]:
text = 'Food taste didnt meet the expectations'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 21ms/step
Prediction value: [1.0000000e+00 1.3267138e-08]
Negative


In [42]:
text = 'I had a pretty day'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 23ms/step
Prediction value: [3.377205e-04 9.996623e-01]
Positive


In [43]:
text = 'The taste was horrendous.'
tester = np.array([text])
tester = pd.DataFrame(tester)
tester.columns = ['Text']

tester['Text'] = tester['Text'].apply(lambda x: x.lower())
tester['Text'] = tester['Text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

max_fatures = 2000
test = tokenizer.texts_to_sequences(tester['Text'].values)
test = pad_sequences(test)

if X.shape[1]>test.shape[1]:
    test = np.pad(test[0], (X.shape[1]-test.shape[1],0), 'constant')

test = np.array([test])

prediction = model.predict(test)
print('Prediction value:',prediction[0])
threshold = 0.5  # You can adjust this threshold based on your model and requirements
prediction_value = prediction[0][0]  # Accessing the first element of the prediction array
if prediction_value <= threshold:
    print("Positive")
else:
    print("Negative")

1/1 [==============================] - 0s 23ms/step
Prediction value: [0.9989071 0.0010929]
Negative
